<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Transforming-Data" data-toc-modified-id="Transforming-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Transforming Data</a></span><ul class="toc-item"><li><span><a href="#Set-up" data-toc-modified-id="Set-up-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Set up</a></span></li><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Loading data</a></span></li><li><span><a href="#CREATION-OF-VARIABLES" data-toc-modified-id="CREATION-OF-VARIABLES-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>CREATION OF VARIABLES</a></span></li><li><span><a href="#REORDENACION-DEL-DATAFRAME" data-toc-modified-id="REORDENACION-DEL-DATAFRAME-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>REORDENACION DEL DATAFRAME</a></span></li></ul></li></ul></div>

# Transforming Data

## Set up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%config IPCompleter.greedy=True
pd.options.display.float_format = '{:15.2f}'.format


## Loading data

In [2]:
df = pd.read_pickle('../Datos/df.pickle')
df

,plant,inverter_id,kw_dc,kw_ac,kw_day,kw_total,sensor_id,t_environment,t_module,irradiation
date,,,,,,,,,,
2020-05-15 00:00:00,p1,1BY6WEcLGh8j5v7,0.00,0.00,0.00,6259559.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00
2020-05-15 00:00:00,p1,1IF53ai7Xc0U56Y,0.00,0.00,0.00,6183645.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00
2020-05-15 00:00:00,p1,3PZuoBAID5Wc2HD,0.00,0.00,0.00,6987759.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00
2020-05-15 00:00:00,p1,7JYdWkrLSPkdwr4,0.00,0.00,0.00,7602960.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00
2020-05-15 00:00:00,p1,McdE0feGgRqW7Ca,0.00,0.00,0.00,7158964.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00
...,...,...,...,...,...,...,...,...,...,...
2020-06-17 23:45:00,p2,q49J1IKaHRwDQnt,0.00,0.00,4157.00,520758.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00
2020-06-17 23:45:00,p2,rrq4fwE8jgrTyWY,0.00,0.00,3931.00,121131356.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00
2020-06-17 23:45:00,p2,vOuJvMaM2sgwLmb,0.00,0.00,4322.00,2427691.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00


## CREATION OF VARIABLES

We begin by extracting the date components and incorporating them as new variables.

In [3]:
def components_dates(dataframe):
    ''' 
    this function returns a dataframe where the components of the date of an initial dataframe.
    
    INPUT:
    dataframe with an index that is an a date
    OUTPUT:
    dataframe with components of these dates
    '''
    month = dataframe.index.month
    day = dataframe.index.day
    hour = dataframe.index.hour
    minute = dataframe.index.minute
    
    return(pd.DataFrame({'month': month, 'day': day, 'hour': hour, 'minute': minute}))

In [4]:
components_dates(df)

,month,day,hour,minute
0,5,15,0,0
1,5,15,0,0
2,5,15,0,0
3,5,15,0,0
4,5,15,0,0
...,...,...,...,...
136467,6,17,23,45
136468,6,17,23,45
136469,6,17,23,45
136470,6,17,23,45


In [5]:
df = pd.concat([df.reset_index(), components_dates(df)], axis = 1).set_index('date')

We are going to create the inverter efficiency variable, which consists of the percentage of DC that transforms to AC satisfactorily.

But we have a very common difficulty in the ratios, that the denominator can be zero.

If this were the case, the ratio would return a null.

In our case the denominator is DC, so if the DC generation was zero the AC generation should be zero as well.

We can correct this by simply imputing the nulls that come out as zeros.


In [6]:
def efficiency_inverter(AC, DC):
    temp = (AC/DC) * 100
    return(temp.fillna(0))

In [7]:
df['efficiency'] = efficiency_inverter(df['kw_ac'], df['kw_dc'])

In [8]:
df

,plant,inverter_id,kw_dc,kw_ac,kw_day,kw_total,sensor_id,t_environment,t_module,irradiation,month,day,hour,minute,efficiency
date,,,,,,,,,,,,,,,
2020-05-15 00:00:00,p1,1BY6WEcLGh8j5v7,0.00,0.00,0.00,6259559.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,1IF53ai7Xc0U56Y,0.00,0.00,0.00,6183645.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,3PZuoBAID5Wc2HD,0.00,0.00,0.00,6987759.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,7JYdWkrLSPkdwr4,0.00,0.00,0.00,7602960.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,McdE0feGgRqW7Ca,0.00,0.00,0.00,7158964.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17 23:45:00,p2,q49J1IKaHRwDQnt,0.00,0.00,4157.00,520758.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00
2020-06-17 23:45:00,p2,rrq4fwE8jgrTyWY,0.00,0.00,3931.00,121131356.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00
2020-06-17 23:45:00,p2,vOuJvMaM2sgwLmb,0.00,0.00,4322.00,2427691.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00


In [9]:
df['efficiency'].describe().T

count         136472.00
mean              25.57
std               40.15
min                0.00
25%                0.00
50%                9.65
75%                9.82
max              100.83
Name: efficiency, dtype: float64

## REORDENACION DEL DATAFRAME

In [10]:
df

,plant,inverter_id,kw_dc,kw_ac,kw_day,kw_total,sensor_id,t_environment,t_module,irradiation,month,day,hour,minute,efficiency
date,,,,,,,,,,,,,,,
2020-05-15 00:00:00,p1,1BY6WEcLGh8j5v7,0.00,0.00,0.00,6259559.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,1IF53ai7Xc0U56Y,0.00,0.00,0.00,6183645.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,3PZuoBAID5Wc2HD,0.00,0.00,0.00,6987759.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,7JYdWkrLSPkdwr4,0.00,0.00,0.00,7602960.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
2020-05-15 00:00:00,p1,McdE0feGgRqW7Ca,0.00,0.00,0.00,7158964.00,HmiyD2TTLFNqkNe,25.18,22.86,0.00,5,15,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17 23:45:00,p2,q49J1IKaHRwDQnt,0.00,0.00,4157.00,520758.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00
2020-06-17 23:45:00,p2,rrq4fwE8jgrTyWY,0.00,0.00,3931.00,121131356.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00
2020-06-17 23:45:00,p2,vOuJvMaM2sgwLmb,0.00,0.00,4322.00,2427691.00,iq8k7ZNt4Mwm3w0,23.20,22.54,0.00,6,17,23,45,0.00


In [11]:
order =['plant','month', 'day', 'hour','sensor_id','irradiation', 't_environment', 't_module', 'inverter_id', 'kw_dc', 'kw_ac', 'efficiency', 'kw_day','kw_total']

In [12]:
df = df[order]
df

,plant,month,day,hour,sensor_id,irradiation,t_environment,t_module,inverter_id,kw_dc,kw_ac,efficiency,kw_day,kw_total
date,,,,,,,,,,,,,,
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,1BY6WEcLGh8j5v7,0.00,0.00,0.00,0.00,6259559.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,1IF53ai7Xc0U56Y,0.00,0.00,0.00,0.00,6183645.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,3PZuoBAID5Wc2HD,0.00,0.00,0.00,0.00,6987759.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,7JYdWkrLSPkdwr4,0.00,0.00,0.00,0.00,7602960.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,McdE0feGgRqW7Ca,0.00,0.00,0.00,0.00,7158964.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,q49J1IKaHRwDQnt,0.00,0.00,0.00,4157.00,520758.00
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,rrq4fwE8jgrTyWY,0.00,0.00,0.00,3931.00,121131356.00
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,vOuJvMaM2sgwLmb,0.00,0.00,0.00,4322.00,2427691.00


## DAILY DATAFRAME

The level of analysis at which we have the data is every 15 minutes, which may be too disaggregated for certain analyses.

We are going to build a version of the dataframe aggregated at the daily level.

For this we use resample for downgrading.

We will need to aggregate by plant and inverter which are the key fields of our dataset.

As we have variables to which different aggregation functions apply, we can use the dictionary format of agg().


In [15]:
df_day = df.groupby(['plant', 'inverter_id']).resample('D') \
    .agg({'irradiation': [min,np.mean,max],
          't_environment': [min,np.mean,max],
          't_module': [min,np.mean,max],
          'kw_dc': [min,np.mean,max,sum],
          'kw_ac': [min,np.mean,max,sum],
          'efficiency': [min,np.mean,max],
          'kw_day': max,
          'kw_total': max})

df_day

irradiation                  \
                                             min            mean   
plant inverter_id     date                                         
p1    1BY6WEcLGh8j5v7 2020-05-15            0.00            0.20   
                      2020-05-16            0.00            0.21   
                      2020-05-17            0.00            0.24   
                      2020-05-18            0.00            0.16   
                      2020-05-19            0.00            0.19   
...                                          ...             ...   
p2    xoJJ8DcxJEcupym 2020-06-13            0.00            0.22   
                      2020-06-14            0.00            0.23   
                      2020-06-15            0.00            0.19   
                      2020-06-16            0.00            0.17   
                      2020-06-17            0.00            0.12   

                                                   t_environment  \
                                             max             min   
plant inverter_id     date                                         
p1    1BY6WEcLGh8j5v7 2020-05-15            0.89           22.04   
                      2020-05-16            0.81           21.50   
                      2020-05-17            1.00           21.21   
                      2020-05-18            0.97           20.96   
                      2020-05-19            0.84           22.39   
...                                          ...             ...   
p2    xoJJ8DcxJEcupym 2020-06-13            0.93           22.20   
                      2020-06-14            0.92           23.65   
                      2020-06-15            0.83           24.00   
                      2020-06-16            0.77           23.63   
                      2020-06-17            0.58           22.55   

                                                                  \
                                            mean             max   
plant inverter_id     date                                         
p1    1BY6WEcLGh8j5v7 2020-05-15           27.43           34.43   
                      2020-05-16           26.78           32.52   
                      2020-05-17           26.69           35.25   
                      2020-05-18           23.85           28.37   
                      2020-05-19           25.34           30.37   
...                                          ...             ...   
p2    xoJJ8DcxJEcupym 2020-06-13           26.12           31.91   
                      2020-06-14           27.02           32.96   
                      2020-06-15           26.56           31.61   
                      2020-06-16           26.37           30.83   
                      2020-06-17           24.85           29.04   

                                        t_module                  \
                                             min            mean   
plant inverter_id     date                                         
p1    1BY6WEcLGh8j5v7 2020-05-15           20.29           32.58   
                      2020-05-16           19.59           31.86   
                      2020-05-17           20.38           32.74   
                      2020-05-18           19.48           27.81   
                      2020-05-19           20.06           29.73   
...                                          ...             ...   
p2    xoJJ8DcxJEcupym 2020-06-13           20.88           30.39   
                      2020-06-14           22.12           31.59   
                      2020-06-15           23.00           30.00   
                      2020-06-16           22.56           29.59   
                      2020-06-17           21.91           26.67   

                                                           kw_dc  ...  \
                                             max             min  ...   
plant inverter_id     date                                        ...   
p1    

It has generated it with multi index, both in rows and columns.

To remove the one in columns we can flatten the names with .to_flat_index(), which is a relatively recent method of Pandas.


In [18]:
tuples = df_day.columns.to_flat_index()
tuples

Index([   ('irradiation', 'min'),   ('irradiation', 'mean'),
          ('irradiation', 'max'),  ('t_environment', 'min'),
       ('t_environment', 'mean'),  ('t_environment', 'max'),
             ('t_module', 'min'),      ('t_module', 'mean'),
             ('t_module', 'max'),          ('kw_dc', 'min'),
               ('kw_dc', 'mean'),          ('kw_dc', 'max'),
                ('kw_dc', 'sum'),          ('kw_ac', 'min'),
               ('kw_ac', 'mean'),          ('kw_ac', 'max'),
                ('kw_ac', 'sum'),     ('efficiency', 'min'),
          ('efficiency', 'mean'),     ('efficiency', 'max'),
               ('kw_day', 'max'),       ('kw_total', 'max')],
      dtype='object')

In [23]:
df_day.columns = ['_'.join(couple) for couple in tuples]
df_day

irradiation_min  irradiation_mean  \
plant inverter_id     date                                            
p1    1BY6WEcLGh8j5v7 2020-05-15             0.00              0.20   
                      2020-05-16             0.00              0.21   
                      2020-05-17             0.00              0.24   
                      2020-05-18             0.00              0.16   
                      2020-05-19             0.00              0.19   
...                                           ...               ...   
p2    xoJJ8DcxJEcupym 2020-06-13             0.00              0.22   
                      2020-06-14             0.00              0.23   
                      2020-06-15             0.00              0.19   
                      2020-06-16             0.00              0.17   
                      2020-06-17             0.00              0.12   

                                  irradiation_max  t_environment_min  \
plant inverter_id     date                                             
p1    1BY6WEcLGh8j5v7 2020-05-15             0.89              22.04   
                      2020-05-16             0.81              21.50   
                      2020-05-17             1.00              21.21   
                      2020-05-18             0.97              20.96   
                      2020-05-19             0.84              22.39   
...                                           ...                ...   
p2    xoJJ8DcxJEcupym 2020-06-13             0.93              22.20   
                      2020-06-14             0.92              23.65   
                      2020-06-15             0.83              24.00   
                      2020-06-16             0.77              23.63   
                      2020-06-17             0.58              22.55   

                                  t_environment_mean  t_environment_max  \
plant inverter_id     date                                                
p1    1BY6WEcLGh8j5v7 2020-05-15               27.43              34.43   
                      2020-05-16               26.78              32.52   
                      2020-05-17               26.69              35.25   
                      2020-05-18               23.85              28.37   
                      2020-05-19               25.34              30.37   
...                                              ...                ...   
p2    xoJJ8DcxJEcupym 2020-06-13               26.12              31.91   
                      2020-06-14               27.02              32.96   
                      2020-06-15               26.56              31.61   
                      2020-06-16               26.37              30.83   
                      2020-06-17               24.85              29.04   

                                    t_module_min   t_module_mean  \
plant inverter_id     date                                         
p1    1BY6WEcLGh8j5v7 2020-05-15           20.29           32.58   
                      2020-05-16           19.59           31.86   
                      2020-05-17           20.38           32.74   
                      2020-05-18           19.48           27.81   
                      2020-05-19           20.06           29.73   
...                                          ...             ...   
p2    xoJJ8DcxJEcupym 2020-06-13           20.88           30.39   
                      2020-06-14           22.12           31.59   
                      2020-06-15           23.00           30.00   
                      2020-06-16           22.56           29.59   
                      2020-06-17           21.91           26.67   

                                    t_module_max       kw_dc_min  ...  \
plant inverter_id     date                                        ...   
p1    1BY6WEcLGh8j5v7 2020-05-15           55.03            0.00  ...   
                      2020-05-16           54.23            0.00  ...   
                    

In [26]:
df_day = df_day.reset_index().set_index('date')

In [27]:
df_day

,plant,inverter_id,irradiation_min,irradiation_mean,irradiation_max,t_environment_min,t_environment_mean,t_environment_max,t_module_min,t_module_mean,...,kw_dc_sum,kw_ac_min,kw_ac_mean,kw_ac_max,kw_ac_sum,efficiency_min,efficiency_mean,efficiency_max,kw_day_max,kw_total_max
date,,,,,,,,,,,,,,,,,,,,,
2020-05-15,p1,1BY6WEcLGh8j5v7,0.00,0.20,0.89,22.04,27.43,34.43,20.29,32.58,...,235340.70,0.00,247.81,1039.35,23046.55,0.00,5.26,9.82,5754.00,6265313.00
2020-05-16,p1,1BY6WEcLGh8j5v7,0.00,0.21,0.81,21.50,26.78,32.52,19.59,31.86,...,256629.88,0.00,285.51,1095.29,25124.49,0.00,5.56,9.83,6292.00,6271605.00
2020-05-17,p1,1BY6WEcLGh8j5v7,0.00,0.24,1.00,21.21,26.69,35.25,20.38,32.74,...,288039.82,0.00,293.47,1114.81,28172.85,0.00,4.99,9.82,7045.00,6278650.00
2020-05-18,p1,1BY6WEcLGh8j5v7,0.00,0.16,0.97,20.96,23.85,28.37,19.48,27.81,...,204030.30,0.00,208.03,1193.63,19970.51,0.00,4.99,9.83,4998.00,6283648.00
2020-05-19,p1,1BY6WEcLGh8j5v7,0.00,0.19,0.84,22.39,25.34,30.37,20.06,29.73,...,232277.27,0.00,244.53,1059.80,22741.18,0.00,4.63,9.83,6449.00,6290097.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-13,p2,xoJJ8DcxJEcupym,0.00,0.22,0.93,22.20,26.12,31.91,20.88,30.39,...,27443.74,0.00,279.59,1239.95,26840.58,0.00,51.91,98.29,6632.00,209312200.00
2020-06-14,p2,xoJJ8DcxJEcupym,0.00,0.23,0.92,23.65,27.02,32.96,22.12,31.59,...,30768.78,0.00,313.43,1328.21,30088.97,0.00,51.94,99.92,7268.00,209319687.00
2020-06-15,p2,xoJJ8DcxJEcupym,0.00,0.19,0.83,24.00,26.56,31.61,23.00,30.00,...,25597.24,0.00,260.93,1256.67,25049.49,0.00,52.94,98.27,7412.67,209325949.00


In [28]:
df

,plant,month,day,hour,sensor_id,irradiation,t_environment,t_module,inverter_id,kw_dc,kw_ac,efficiency,kw_day,kw_total
date,,,,,,,,,,,,,,
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,1BY6WEcLGh8j5v7,0.00,0.00,0.00,0.00,6259559.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,1IF53ai7Xc0U56Y,0.00,0.00,0.00,0.00,6183645.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,3PZuoBAID5Wc2HD,0.00,0.00,0.00,0.00,6987759.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,7JYdWkrLSPkdwr4,0.00,0.00,0.00,0.00,7602960.00
2020-05-15 00:00:00,p1,5,15,0,HmiyD2TTLFNqkNe,0.00,25.18,22.86,McdE0feGgRqW7Ca,0.00,0.00,0.00,0.00,7158964.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,q49J1IKaHRwDQnt,0.00,0.00,0.00,4157.00,520758.00
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,rrq4fwE8jgrTyWY,0.00,0.00,0.00,3931.00,121131356.00
2020-06-17 23:45:00,p2,6,17,23,iq8k7ZNt4Mwm3w0,0.00,23.20,22.54,vOuJvMaM2sgwLmb,0.00,0.00,0.00,4322.00,2427691.00


In [30]:
df.to_pickle('../Datos/df.pickle')
df_day.to_pickle('../Datos/df_day.pickle')